# User Interaction of Deep Impact

This Jupyter notebook is designed to establish a **user interface** that allows users to modify the parameters of an asteroid by **dragging sliders** or **entering values directly**. 

The purpose is to observe the hazard of small asteroidshow under varying conditions. It presents the number of population affected, and displays the affected areas and the trajectory of the asteroid on a map.

Please try reasonable values.

Users can also input **a CSV file** containing various scenarios (default file path is './resources/impact_parameter_list.csv'). This will plot the damage zones of each impact on a map and create a heatmap showing the probability of a given postcode being affected by the impact events.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, HTML
import os

In [2]:
from deepimpact import solver, damage, mapping, locator

In [3]:
planet = solver.Planet()
loc = locator.GeospatialLocator()
def update_parameters(radius, velocity, density, strength, angle, latitude, longitude, bearing):
    result = planet.solve_atmospheric_entry(radius, velocity, density, strength, angle)
    energy = planet.calculate_energy(result)
    outcome = planet.analyse_outcome(energy)
    try:
        zone = damage.damage_zones(outcome, latitude, longitude, bearing, pressures = [30000])
    except:
        display(HTML("<b style='color: red;'>PLEASE USE REASONABLE VALUES!!!!!</b>"))

    # to display the postcodes affected
    # postcodes_affected = loc.get_postcodes_by_radius((zone[0],zone[1]), zone[2])
    # display(postcodes_affected)
    
    pop_affected = loc.get_population_by_radius((zone[0],zone[1]), zone[2])
    # if you want to see the tail just set it as True
    map_affected = mapping.plot_meteor(zone[0], zone[1], zone[2],latitude, longitude, tail = False)
    display(HTML(f"<b style='color: red;'>Population affected: {pop_affected[0]}</b>"))
    display(map_affected)

In [4]:
interact(update_parameters,
         radius = widgets.FloatSlider(min=1, max=1e2, value=34),
         velocity=widgets.FloatSlider(min=1, max=1e5, value=18723),
         density=widgets.FloatSlider(min=1, max=1e5, value=3123),
         strength=widgets.FloatSlider(min=1, max=1e8, value=12836319),
         angle=widgets.FloatSlider(min=25, max=90, value=44),
         latitude = widgets.FloatSlider(min=49, max=61,value=53),
         longitude = widgets.FloatSlider(min=-10, max=3,value=-2.48),
         bearing = widgets.FloatSlider(min=0, max=200,value=115))

interactive(children=(FloatSlider(value=34.0, description='radius', min=1.0), FloatSlider(value=18723.0, descr…

<function __main__.update_parameters(radius, velocity, density, strength, angle, latitude, longitude, bearing)>

In [5]:
def multi_scenario(impact_file):
    meteors = mapping.plot_meteors(tail=True,
                                   filename=impact_file)
    probability, population = damage.impact_risk(planet = solver.Planet(),
                                                  impact_file=impact_file)
    heatmap = mapping.plot_heatmap(probability, population,
                           latlongfile='./resources/full_postcodes.csv')
    display(meteors)
    display(heatmap)


interact(multi_scenario,
         impact_file = widgets.Text(value='./resources/impact_parameter_list.csv'),
                                    placeholder='Please enter the file path', 
                                    description='File Path:',disabled=False)

interactive(children=(Text(value='./resources/impact_parameter_list.csv', description='impact_file'), Output()…

<function __main__.multi_scenario(impact_file)>